In [1]:
import numpy as np
from python.py_rle import mask2rle, rle2mask 
from cython_rle import cy_mask_to_rle
from py_multi_backend import (
    arr2rle, 
    c_order_mask2rle, 
    f_order_mask2rle, 
    thread_arr2rle, 
    thread_f_order_mask2rle, 
    thread_c_order_mask2rle,
    rle2mask as rust_rle2mask
)

import shapely
from pycocotools import mask as cocomask
from rasterio.features import rasterize
from shapely.geometry import Polygon, Point

In [2]:
SIZE = (2048, 2048)
rand_mask = np.random.random(SIZE) < 0.5
small_mask = rasterize(
    [shapely.affinity.translate(Polygon([(-32, -16), (0, 16), (32, -16)]), xoff=SIZE[0] / 2, yoff=SIZE[1] / 2)], 
    out_shape=SIZE,
).astype(bool)
static_mask = np.ones(SIZE) < 0.5

rand_rle = arr2rle(rand_mask.ravel("F"))
static_rle = arr2rle(static_mask.ravel("F"))
small_rle = arr2rle(small_mask.ravel("F"))

static_seg_rle = cocomask.encode(np.asfortranarray(static_mask))
static_seg_rle["counts"] = static_seg_rle["counts"].decode("utf-8")
rand_seg_rle = cocomask.encode(np.asfortranarray(rand_mask))
rand_seg_rle["counts"] = rand_seg_rle["counts"].decode("utf-8")
small_seg_rle = cocomask.encode(np.asfortranarray(small_mask))
small_seg_rle["counts"] = small_seg_rle["counts"].decode("utf-8")

### RLE to Mask Benchmarks

In [3]:
assert (rust_rle2mask(static_rle, SIZE) == static_mask).all()
assert (rust_rle2mask(rand_rle, SIZE) == rand_mask).all()
assert (rust_rle2mask(small_rle, SIZE) == small_mask).all()

In [4]:
%%timeit
_ = rust_rle2mask(static_rle, SIZE)

528 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
%%timeit
_ = rust_rle2mask(rand_rle, SIZE)

69.2 ms ± 269 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
_ = rust_rle2mask(small_rle, SIZE)

521 µs ± 670 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit
_ = rle2mask(static_seg_rle)

5.39 ms ± 22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
_ = rle2mask(rand_seg_rle)

39.9 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
_ = rle2mask(small_seg_rle)

5.46 ms ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Mask to RLE Benchmarks

In [10]:
assert cy_mask_to_rle(rand_mask)["counts"] == mask2rle(rand_mask)["counts"]
assert arr2rle(rand_mask.ravel(order="F")) == mask2rle(rand_mask)["counts"]
assert f_order_mask2rle(rand_mask) == mask2rle(rand_mask)["counts"]

In [11]:
%%timeit
_ = arr2rle(rand_mask.ravel("F"))

51 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
_ = arr2rle(static_mask.ravel("F"))

15.9 ms ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
_ = f_order_mask2rle(rand_mask)

44.3 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
_ = f_order_mask2rle(static_mask)

12.7 ms ± 34.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
_ = c_order_mask2rle(rand_mask)

38.3 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
_ = c_order_mask2rle(static_mask)

2.98 ms ± 54.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
_ = cy_mask_to_rle(rand_mask)

121 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
_ = cy_mask_to_rle(static_mask)

85.4 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
_ = mask2rle(rand_mask)

6.71 s ± 67.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
_ = mask2rle(static_mask)

132 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
